In [36]:
import numpy as np
import pandas as pd
import seaborn as sns

In [67]:
# Import my modules.
import sys, os
from pathlib import Path
current_dir = os.path.join(Path().resolve())
sys.path.append(str(current_dir) + '/../')

from modules import utils
from modules import models
from modules import preprocess

import importlib
for m in [utils, models, preprocess]:
    importlib.reload(m)

In [63]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
mbd = 'microbusiness_density'

## Trend実験

In [61]:
mbd = 'microbusiness_density'

df_train, df_test, df_subm = utils.load_dataset()
df_all, df_census = utils.merge_dataset(df_train, df_test, pop=False, unemploy=False, census=False, coord=True, co_est=False, fix_pop=False, add_location=False, outlier=False)
# df_all = preprocess.add_lag_features(df_all)

/tmp/working/code-analysis/../modules/utils.py:321: DtypeWarning: Columns (102) have mixed types.Specify dtype option on import or set low_memory=False.
  df_census = load_census(BASE)


In [68]:
df_all = preprocess.add_location(df_all)

In [40]:
low_trend_params = {
    1: {
        'n':3,
        'thre':3,
        'thre_r':0,
        'lower_bound': 60,
        'upper_bound': 140,
        'use_regularize': True,
        'v_regularize': [0.03, 0.02],
        'v_clip':[0.999, 1.004]
    }
}
params = low_trend_params[1]
df_trend, trend_dict = preprocess.get_trend_dict(df_all, **params)
print(len(trend_dict))
trend_dict

13


{13025: 1.004,
 13193: 0.999,
 18025: 0.999,
 21147: 1.004,
 31093: 1.0033846255390906,
 40005: 1.004,
 40069: 1.00303696951699,
 46091: 1.0016915440454517,
 48305: 1.004,
 48385: 1.0026579234749724,
 5067: 1.004,
 54023: 0.999,
 8033: 1.004}

In [41]:
high_trend_params = {
    1: {
        'n':3,
        'thre':3,
        'thre_r':0.004,
        'lower_bound': 20000,
        'upper_bound': 99999999,
        'use_regularize': True,
        'v_regularize': [0.002, 0],
        'v_clip':[None, None]
    }
}
params = high_trend_params[1]
df_trend, high_trend_dict = preprocess.get_trend_dict(df_all, **params)
print(len(high_trend_dict))
high_trend_dict

20


{12031: 1.0086328445657107,
 12101: 1.006481142747778,
 17197: 1.0080206078421285,
 20091: 1.0060327304334955,
 25025: 1.0061688635360524,
 36085: 1.0173209335038416,
 37119: 1.006098418566488,
 40109: 1.0129319463023307,
 40143: 1.008331800893501,
 42003: 1.0233533145463996,
 47065: 1.008465009610966,
 47187: 1.0075341777021485,
 48113: 1.0045559762608431,
 48121: 1.0118033817358585,
 48157: 1.013289253684673,
 48491: 1.0074687019144728,
 55133: 1.0069273342745635,
 6001: 1.0116187765330433,
 6073: 1.0100269518719414,
 6079: 1.0050638864187709}

In [42]:
middle_trend_params = {
    1: {
        'n':7,
        'thre':7,
        'thre_r':0.001,
        'lower_bound': 1000,
        'upper_bound': 20000,
        'use_regularize': True,
        'v_regularize': [0.001, 0],
        'v_clip':[None, 1.02]
    }
}
params = middle_trend_params[1]
df_trend, middle_trend_dict = preprocess.get_trend_dict(df_all, **params)
print(len(middle_trend_dict))
middle_trend_dict

29


{10001: 1.0129351534528772,
 1095: 1.008447263901533,
 1125: 1.0154674931169532,
 13285: 1.02,
 16021: 1.02,
 16051: 1.0102600596450686,
 17049: 1.0145852062314247,
 17119: 1.0160684278423038,
 18073: 1.02,
 23017: 1.02,
 24023: 1.02,
 29209: 0.9900118427149365,
 30047: 1.0077290823975364,
 30063: 1.007380223819338,
 32017: 0.989911033357664,
 33009: 1.0036445676092536,
 37071: 1.0116879058571973,
 37097: 1.008211996600611,
 37161: 1.0085466949373592,
 37175: 1.0167317606498527,
 37195: 1.0099083923055931,
 42013: 0.984723291353719,
 46083: 1.008549269096111,
 49057: 1.0060150299560793,
 51033: 1.0088285129605443,
 53007: 1.02,
 53065: 0.9778049488905132,
 6005: 0.9940137900855888,
 8037: 1.0155565410397998}

In [43]:
lowmiddle_trend_params = {
    1: {
        'n':10,
        'thre':10,
        'thre_r':0.003,
        'lower_bound': 200,
        'upper_bound': 1000,
        'use_regularize': True,
        'v_regularize': [0.001, 0],
        'v_clip':[None, 1.05]
    }
}
params = lowmiddle_trend_params[1]
df_trend, lowmiddle_trend_dict = preprocess.get_trend_dict(df_all, **params)
print(len(lowmiddle_trend_dict))
lowmiddle_trend_dict

2


{13133: 1.0124139778496224, 17135: 1.05}

In [44]:
df_sub1 = pd.read_csv('../submission/submission_13738.csv', index_col='row_id')
df_sub1, df_extract, var_dict = utils.insert_trend(df_sub1, df_all, df_census, trend_dict)
df_sub1, df_extract, var_dict = utils.insert_trend(df_sub1, df_all, df_census, high_trend_dict, method='replace')
df_sub1, df_extract, var_dict = utils.insert_trend(df_sub1, df_all, df_census, middle_trend_dict, method='replace')
df_sub1, df_extract, var_dict = utils.insert_trend(df_sub1, df_all, df_census, lowmiddle_trend_dict, method='replace')
df_sub1.to_csv('../submission/submission_13738_trend.csv')

In [45]:
df_merged = utils.compare_submission(df_sub1, 'submission_13738')
df_diff = df_merged[df_merged['smape']>0.001].copy()
df_diff.shape

(63, 6)

In [46]:
df_merged['smape'].sum()

43.348071578644145

In [26]:
df_merged[df_merged['smape']>0]

,row_id,microbusiness_density,cfips,month,baseline,smape
47,1095_2023-01-01,2.826422,1095,2023-01-01,2.816695,0.344734
62,1125_2023-01-01,4.027637,1125,2023-01-01,3.987585,0.999398
142,5067_2023-01-01,0.713956,5067,2023-01-01,0.711111,0.399202
184,6001_2023-01-01,11.784161,6001,2023-01-01,11.705370,0.670857
186,6005_2023-01-01,13.309681,6005,2023-01-01,13.435769,0.942877
220,6073_2023-01-01,15.107975,6073,2023-01-01,15.032598,0.500167
223,6079_2023-01-01,11.868999,6079,2023-01-01,11.864585,0.037191
259,8033_2023-01-01,3.934325,8033,2023-01-01,3.918651,0.399202
261,8037_2023-01-01,14.962432,8037,2023-01-01,14.798017,1.104928
314,10001_2023-01-01,7.155430,10001,2023-01-01,7.100794,0.766493


In [71]:
df_submission = pd.read_csv('../submission/submission_2023-03-05_17_36_11.csv')
sub = pd.read_csv('../submission/submission_lb_13803.csv')
for i, row in sub.iterrows():
    df_submission.iat[i,1] = (
        0.3*df_submission.iat[i,1] +
        0.7*row["microbusiness_density"]
    )

df_submission = df_submission.set_index('row_id')
df_merged = utils.compare_submission(df_submission, 'submission_13769')
print(df_merged['smape'].mean())

0.7893422334053801


In [72]:
df_submission = pd.read_csv('../submission/submission_2023-03-05_18_13_38.csv')
sub = pd.read_csv('../submission/submission_lb_13803.csv').rename(columns={mbd:'mbd_pred'})
df_submission = df_submission.merge(sub, how='left', on='row_id')
df_submission[mbd] = df_submission[mbd] * 0.3 + df_submission['mbd_pred'] * 0.7
df_submission = df_submission.drop(['mbd_pred'], axis=1).set_index('row_id')

df_merged = utils.compare_submission(df_submission, 'submission_13769')
print(df_merged['smape'].mean())

0.005707197795724607


In [28]:
# Import my modules.
import sys, os
from pathlib import Path
current_dir = os.path.join(Path().resolve())
sys.path.append(str(current_dir) + '/../')

from modules import utils
from modules import models
from modules import preprocess

import importlib
for m in [utils, models, preprocess]:
    importlib.reload(m)

In [29]:
df_allt = df_all.copy()
df_allt = utils.merge_scale41(df_allt, df_submission, df_census)

In [30]:
df_allt[(df_allt['scale']>=38)&(df_allt['scale']<=41)].head(100)

,cfips,county,state,microbusiness_density,active,year,month,scale,state_i,mbd_origin,select_lastactive30,select_lastmbd30,select_lastactive31,select_lastmbd31,select_lastactive32,select_lastmbd32,select_lastactive33,select_lastmbd33,select_lastactive34,select_lastmbd34,select_lastactive35,select_lastmbd35,select_lastactive36,select_lastmbd36,select_lastactive37,select_lastmbd37,select_lastactive38,select_lastmbd38,select_lastactive39,select_lastmbd39,select_lastactive40,select_lastmbd40,select_rate1,select_active_lag1,select_mbd_lag1,select_mbd_origin_lag1,select_active_lag2,select_mbd_lag2,select_mbd_origin_lag2,select_active_lag3,select_mbd_lag3,select_mbd_origin_lag3,select_active_lag4,select_mbd_lag4,select_mbd_origin_lag4,select_active_lag5,select_mbd_lag5,select_mbd_origin_lag5,select_active_lag6,select_mbd_lag6,select_mbd_origin_lag6,select_active_lag7,select_mbd_lag7,select_mbd_origin_lag7,select_active_lag1_diff1,select_active_lag1_diff2,select_active_lag1_diff3,select_active_lag1_diff4,select_active_lag1_diff5,select_active_lag1_diff6,select_active_lag1_diff7,select_rate1_lag1,select_rate1_lag2,select_rate1_lag3,select_rate1_lag4,select_rate1_lag5,select_rate1_lag6,select_rate1_lag7,select_rate1_rsum2,select_rate1_rsum4,select_rate1_rsum6,select_rate1_rsum8,select_rate1_rsum10,select_rate1_rsum12
row_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10001_2022-10-01,10001,0,Delaware,6.969616,9618.0,2022,10,38,0,6.969616,8766.0,6.352220,8895.0,6.445699,8973.0,6.502221,8945.0,6.481931,9126.0,6.613092,9420.0,6.826137,9458.0,6.853673,9486.0,6.873963,9618.0,6.969616,9708.0,7.034833,9784.0,7.089906,0.013915,9486.0,6.873963,6.873963,9458.0,6.853673,6.853673,9420.0,6.826137,6.826137,9126.0,6.613092,6.613092,8945.0,6.481931,6.481931,8973.0,6.502221,6.502221,8895.0,6.445699,6.445699,28.0,66.0,360.0,541.0,513.0,591.0,720.0,0.002960,0.004034,0.032216,0.020235,-0.003120,0.008769,0.014716,6.994407e-03,0.059445,0.065093,0.101369,0.104133,0.109016
10001_2022-11-01,10001,0,Delaware,7.034833,9708.0,2022,11,39,0,7.034833,8766.0,6.352220,8895.0,6.445699,8973.0,6.502221,8945.0,6.481931,9126.0,6.613092,9420.0,6.826137,9458.0,6.853673,9486.0,6.873963,9618.0,6.969616,9708.0,7.034833,9784.0,7.089906,0.009357,9618.0,6.969616,6.969616,9486.0,6.873963,6.873963,9458.0,6.853673,6.853673,9420.0,6.826137,6.826137,9126.0,6.613092,6.613092,8945.0,6.481931,6.481931,8973.0,6.502221,6.502221,132.0,160.0,198.0,492.0,673.0,645.0,723.0,0.013915,0.002960,0.004034,0.032216,0.020235,-0.003120,0.008769,1.687570e-02,0.053125,0.070240,0.093725,0.109151,0.121620
10001_2022-12-01,10001,0,Delaware,7.089906,9784.0,2022,12,40,0,7.089906,8766.0,6.352220,8895.0,6.445699,8973.0,6.502221,8945.0,6.481931,9126.0,6.613092,9420.0,6.826137,9458.0,6.853673,9486.0,6.873963,9618.0,6.969616,9708.0,7.034833,9784.0,7.089906,0.007829,9708.0,7.034833,7.034833,9618.0,6.969616,6.969616,9486.0,6.873963,6.873963,9458.0,6.853673,6.853673,9420.0,6.826137,6.826137,9126.0,6.613092,6.613092,8945.0,6.481931,6.481931,90.0,222.0,250.0,288.0,582.0,763.0,735.0,0.009357,0.013915,0.002960,0.004034,0.032216,0.020235,-0.003120,2.327267e-02,0.030267,0.082717,0.088366,0.124641,0.127406
10001_2023-01-01,10001,0,Delaware,7.094430,9790.0,2023,1,41,0,7.094430,8766.0,6.352220,8895.0,6.445699,8973.0,6.502221,8945.0,6.481931,9126.0,6.613092,9420.0,6.826137,9458.0,6.853673,9486.0,6.873963,9618.0,6.969616,9708.0,7.034833,9784.0,7.089906,0.000000,9784.0,7.089906,7.089906,9708.0,7.034833,7.034833,9618.0,6.969616,6.969616,9486.0,6.873963,6.873963,9458.0,6.853673,6.853673,9420.0,6.826137,6.826137,9126.0,6.613092,6.613092,76.0,166.0,298.0,326.0,364.0,658.0,839.0,0.007829,0.009357,0.013915,0.002960,0.004034,0.032216,0.020235,1.718599e-02,0.034062,0.070311,0.087426,0.110911,0.126337
10003_2022-10-01,10003,1,Delaware,20.917912,91633.0,2022,10,38,0,20.917912,75793.0,17.301968,77514.0,17.694836,83150.0,18.981419,82553.0,18.845135,82302.0,18.787838,82490.0,18.830753,87972.0,20.082180,90367.0,20.6